# [Linear regression using StatsModels](https://medium.com/evidentebm/linear-regression-using-statsmodels-d0db5fef16bb)

Aim: to work my way through this example and learn

In [4]:
import numpy as np 
import pandas as pd
import statsmodels.formula.api as smf
df = pd.read_csv("./data/framingham.csv")
print("shape of df:", df.shape)
print("Columns: ", df.columns)
df.head()

shape of df: (4238, 16)
Columns:  Index(['male', 'age', 'education', 'currentSmoker', 'cigsPerDay', 'BPMeds',
       'prevalentStroke', 'prevalentHyp', 'diabetes', 'totChol', 'sysBP',
       'diaBP', 'BMI', 'heartRate', 'glucose', 'TenYearCHD'],
      dtype='object')


,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [5]:
df[['male', 'age', 'education', 'sysBP']].describe()

,male,age,education,sysBP
count,4238.000000,4238.000000,4133.000000,4238.000000
mean,0.429212,49.584946,1.978950,132.352407
std,0.495022,8.572160,1.019791,22.038097
min,0.000000,32.000000,1.000000,83.500000
25%,0.000000,42.000000,1.000000,117.000000
50%,0.000000,49.000000,2.000000,128.000000
75%,1.000000,56.000000,3.000000,144.000000
max,1.000000,70.000000,4.000000,295.000000


# Research question

Is there an association between systolic blood pressure and age, after adjusting for relevant confounders?

 * continuous outcome: (systolic blood pressure in mmHg) 
 * independent continuous covariable (age in years)
 
We will consider the following available variables as potential relevant confounders for the association between sBP and age:

 * gender (binary variable: male), 
 * education (categorical variable: education), 
 * smoking (continuous variable: cigsPerDay for cigarettes per day), 
 * intake of medication for hypertension (binary variable: BPMeds), 
 * total cholesterol (continuous variable: totChol).
 
Defining the null hypothesis

 * Null hypothesis (H0): There is NO association between sBP and age (considered the chosen confounders);
 * Alternative hypothesis (H1): There is an association between sBP and age (considered the chosen confounders).

In [6]:
model1 = smf.ols(formula='sysBP ~ age', data=df).fit()

   * smf calls the package Statsmodel
   *  ols tells Python we are using an Ordinary Least Square (OLS) regression (a type linear regression)
   * formula= used to write the dependent and all the independent variable(s)
   *  first variable inside the parenthesis before “~”/dependent variable/outcome: The first variable is our only dependent variable. This is our outcome, the variable that determines which type of regression to use, and the one to be associated with all other covariates;
  *   ~ inside parenthesis: Marks the border between the outcome (dependent variable) to the left, and the covariates (independent variables) to the right;
   *  independent covariates/independent variables: All other variables after the “~”, inside parenthesis;
   *  + sign inside parenthesis: the + sign is used to separate different independent variables inside the same model (useful for multivariable models, aka many independent variables)
   *  ,data=: This marks the name of the data frame.
   *  .fit() tells Python we want to fit our function (“run the function”)

In [7]:
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:                  sysBP   R-squared:                       0.155
Model:                            OLS   Adj. R-squared:                  0.155
Method:                 Least Squares   F-statistic:                     779.8
Date:                Mon, 06 Feb 2023   Prob (F-statistic):          1.14e-157
Time:                        08:19:44   Log-Likelihood:                -18762.
No. Observations:                4238   AIC:                         3.753e+04
Df Residuals:                    4236   BIC:                         3.754e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     82.0878      1.827     44.939      0.0

Question:
    
    1. what is R-squre of 16% mean
    2. what is f-statistics? why important
    3. what is t-ratio
    4. explain the confidence interval once more
    5. what this mean? "Before reading the output, we see this is a valid model but… it is not quite what we wanted… Discrete variables with multiple categories (eg; education) are not stratified by groups." 
    6. what happens if I control for male and female
    7. what does control mean? 

In [8]:
model2 = smf.ols(formula='sysBP ~ age + male + education + cigsPerDay + BPMeds + totChol', 
                 data=df).fit() 
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                  sysBP   R-squared:                       0.213
Model:                            OLS   Adj. R-squared:                  0.212
Method:                 Least Squares   F-statistic:                     180.5
Date:                Mon, 06 Feb 2023   Prob (F-statistic):          7.05e-204
Time:                        08:40:19   Log-Likelihood:                -17587.
No. Observations:                4005   AIC:                         3.519e+04
Df Residuals:                    3998   BIC:                         3.523e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     81.0708      2.449     33.098      0.0

In [9]:
model3 = smf.ols(formula='sysBP ~ age + male + C(education) + cigsPerDay + BPMeds + totChol', 
                 data=df).fit()
print(model3.summary())

                            OLS Regression Results                            
Dep. Variable:                  sysBP   R-squared:                       0.214
Model:                            OLS   Adj. R-squared:                  0.212
Method:                 Least Squares   F-statistic:                     135.6
Date:                Mon, 06 Feb 2023   Prob (F-statistic):          5.20e-202
Time:                        08:44:16   Log-Likelihood:                -17586.
No. Observations:                4005   AIC:                         3.519e+04
Df Residuals:                    3996   BIC:                         3.525e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              78.9177    

In [10]:
model3 = smf.ols(formula='sysBP ~ age + C(male) + C(education) + cigsPerDay + BPMeds + totChol', 
                 data=df).fit()
print(model3.summary())

                            OLS Regression Results                            
Dep. Variable:                  sysBP   R-squared:                       0.214
Model:                            OLS   Adj. R-squared:                  0.212
Method:                 Least Squares   F-statistic:                     135.6
Date:                Mon, 06 Feb 2023   Prob (F-statistic):          5.20e-202
Time:                        08:46:57   Log-Likelihood:                -17586.
No. Observations:                4005   AIC:                         3.519e+04
Df Residuals:                    3996   BIC:                         3.525e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              78.9177    

In [11]:
!pwd


/Users/bxb/Documents/work/0_morning_fun_7-9/morning_fun_7-9/linear_regression_ols
